In [1]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
def get_model_performance(X, y, model_func, epochs=100):
    full_history =  []
    confusion_matrices= []
    for i in range(len(X)):
        X_test, y_test = X[i], y[i]
        X_train, y_train = [ X[j] for j in range(len(X)) if j != i], [y[j] for j in range(len(X)) if j != i]
        X_train, y_train = np.vstack(X_train), np.hstack(y_train)
        
        X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
        y_train = y_train.reshape(y_train.shape[0],1)
        X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
        y_test = y_test.reshape(y_test.shape[0], 1)
        y_train = y_train.astype(np.int64)
        y_test = y_test.astype(np.int64)
        model = model_func(X_train)
        history = model.fit(X_train, y_train, epochs= epochs, validation_data = (X_test, y_test))
        full_history.append(history)
        pred = model.predict(X_test)
        pred = np.argmax(pred, axis=1)
        confusion_matrix = metrics.confusion_matrix(pred, y_test)
        confusion_matrices.append(confusion_matrix)

    return full_history, confusion_matrices

In [14]:
def CNN_Model(data):
    model = keras.Sequential(name="CNN_Sequential")
    model.add(layers.Input(shape = data.shape[1:]))
    model.add(layers.Conv1D(32, 3, activation='relu', name='conv_1'))
    model.add( layers.Conv1D(16, 4, activation="softmax", name="conv_2") )
    model.add(layers.Flatten())
    model.add( layers.Dense(32, activation="relu", name="dense_1") )
    model.add( layers.Dense(64, activation="relu", name="dense_2") )
    model.add( layers.Dense(5, activation="softmax", name="output") )
    model.compile(optimizer=tf.keras.optimizers.SGD(), loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [16]:
X1 = np.load("../COPD_dataset/hold-out-validation/cp_raw_data.npy")
X2 = np.load("../COPD_dataset/hold-out-validation/md1_raw_data.npy")
X3 = np.load("../COPD_dataset/hold-out-validation/md2_raw_data.npy")
X4 = np.load("../COPD_dataset/hold-out-validation/vb_raw_data.npy")

y = [X1[:,-1],X2[:,-1],X3[:,-1],X4[:,-1],]

X = [X1[:,:-1], X2[:,:-1], X3[:,:-1], X4[:,:-1]]

In [17]:
cnn_history, cnn_confusion_matrices = get_model_performance(model_func= CNN_Model, X= X, y= y, epochs=100)

Model: "CNN_Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv1D)             (None, 71998, 32)         128       
                                                                 
 conv_2 (Conv1D)             (None, 71995, 16)         2064      
                                                                 
 flatten_8 (Flatten)         (None, 1151920)           0         
                                                                 
 dense_1 (Dense)             (None, 32)                36861472  
                                                                 
 dense_2 (Dense)             (None, 64)                2112      
                                                                 
 output (Dense)              (None, 5)                 325       
                                                                 
Total params: 36,866,101
Trainable params: 36,866,10

KeyboardInterrupt: 

In [ ]:
def save_history(filename, history):
    np_history = []
    for i in range(len(history)):
        his = history[i]
        np_history.append(his.history)
    np.save(filename, np_history)

In [9]:
save_history("./history,FDDLM_100.npy", fddlm_history)

NameError: name 'fddlm_history' is not defined